In [ ]:
import numpy as np
import torch
import sys

from keras.utils import to_categorical

import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
sns.set()

sys.path.append('../')
import util

model = torch.load('../results/benchmark/SENet_nopool/model_12.pkl', map_location=torch.device('cpu')).double()
proteins = 'XACDEFGHIKLMNPQRSTVWY'


In [ ]:
l = 100

a = 'GIGDPVTCLKSGAICHPVFCPRRYKQIGTCGLPGTKCCKKP'
#a = 'FFDEKCNKLKGTCKNNCGKNEELIALCQKSLKCCRTIQPSGSIID'
#a = 'ACYCRIPACLAGERRYGTCFYRRRVWAFCC'

b = [proteins.index(i) for i in a]
raw = np.array([0]*(l-len(b))+b)

x = to_categorical(raw, num_classes=21)

raw = torch.from_numpy(raw).int()
x = torch.from_numpy(x).double()
print(x.size())
    
x = x.reshape(1, l, 21)
s = np.where(raw!=0)[0][0]

x = x.permute(0, 2, 1)
x1 = model.layer1(x) # n*128*L
print(x1.size())

x2 = model.drop(x1) # n*128*(L/5)
x2 = model.layer2(x2) # n*128*(L/5)
print(x2.size())

x3 = model.drop(x2) #n*128*(L/5)
x3 = model.layer3(x3) # n*64*(L/5)
print(x3.size())

x4 = model.drop(x3) # n*64*(L/5)
x4 = model.layer4(x4) # n*64*(L/5)
print(x4.size())

out = x4.reshape(64, l).detach().numpy()
print(out.shape)
w = model.fc.weight[1].detach().numpy()
p = np.array([w[i]*out[i] for i in range(out.shape[0])])
q = np.array([np.mean(p[:, i]) for i in range(p.shape[1])])[s:]

plt.figure(figsize=(20, 1))
protein = [proteins[i] for i in raw[s:]]
img = sns.heatmap(q.reshape(1, -1), cmap='Blues', xticklabels=False, yticklabels=False,
                  annot=np.array(protein).reshape(1, -1), fmt='')